# Homework 7: Model Selection

**Due**: Monday May 29th

- **Format**: We expect students to complete the homework notebooks using Google Colab (see Discussion 1), but this is not explicitly required and you may use whatever software you would like to run notebooks. 
- **Answers**: As a general guiding policy, you should always try to make it as clear as possible what your answer to each question is, and how you arrived at your answer. Generally speaking, this will mean including all code used to generate results, outputting the actual results to the notebook, and (when necessary) including written answers to support your code.
- **Submission**: Homeworks will be *submitted to Gradescope*, and we expect all students to do question matching on Gradescope upon submission.
- **Late Policy**: All students are allowed 7 total slip days for the quarter, and at most 5 can be used for a single HW assignment. There will be no late credit if you have used up all your slip days. Also, your lowest HW grade will be dropped.

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
sns.set_style()

## Question 1: Identifying Bias and Variance

For each of the comparisons below, identify which model has higher bias and which model has higher variance. Explain your reasoning.

**Part (a)**: Suppose $y \sim \beta_0 + \beta_1 x + \epsilon$, where $\epsilon \sim N(0,\sigma^2)$. 

We train two models on a sample of $n$ randomly generated $y_i$'s.
1. The first model is fit via least squares and has the fitted form $\hat{y} = \hat{\beta}_0$. 
2. The second model has the fitted form $\hat{y} = y_1$.

*Answer*: Model 1 has lower bias since it is trained on all the data versus a single data point, so it will fit the training data more closely. 
Model 1 also has lower variance since it's based on an average of all the data points, which will vary less across datasets than the value of the first data point. 

**Part (b)**: Suppose we have the same data as in part (a).

We fit two models via least squares:
1. The first model's fitted form is $\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x + \hat{\beta}_2 x^2 + \cdots + \hat{\beta}_{n-1} x^{n-1}$.
2. The second model's fitted form is $\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x + \hat{\beta}_2 x^2 + \cdots + \hat{\beta}_{n-2} x^{n-2}$.

*Answer*: Model 1 has lower bias since it will perfectly fit every point in the sample (i.e., same number of $\hat{\beta}$ parameters as data points).
Model 2 has lower variance because we're afforded one additional degree of freedom in the fit, and will generalize slightly better than the first model. Note, however, that both of these models are horrendously overfitted!

**Part (c)**: Now suppose $y \sim \beta_0 + \beta_1 x + \beta_2 x^2 + \epsilon$, where $\epsilon \sim N(0,\sigma^2)$. Further suppose $z \sim \beta_0 + \beta_1 x + \beta_2 x^2 + \beta_3 x^3 + \epsilon$, where $\epsilon \sim N(0,\sigma^2)$.

We fit two models via least squares:
1. The first model's fitted form is $\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x + \hat{\beta}_2 x^2 + \hat{\beta}_3 x^3$.
2. The second model's fitted form is $\hat{z} = \hat{\beta}_0 + \hat{\beta}_1 x + \hat{\beta}_2 x^2$.

*Answer*: Model 1 is unbiased since the functional form of the true data generating process is contained in the model specification.
This is not the case for model 2.
However, model 1 has higher variance than model 2 since it is estimating more parameters than the true DGP, and is therefore fitting to noise more strongly than model 2.

## Question 2: Bias and Variance of Linear Regression

In this question, we will explore the bias-variance tradeoff for a simple linear regression model.

**Part (a)**: Consider simple linear regression data $(x_1, y_1), \ldots, (x_n, y_n)$ that has been standardized so that $\sum_i x_i = 0, \sum_i x_i^2 = 1, \sum_i y_i = 0, \sum_i y_i^2 = 1$. What are the resulting least squares estimates for $\beta_0$ and $\beta_1$ on this standardized data?

*Hint*: You may start from the formula's for $\beta_1$ and $\beta_0$ that are in the lecture notes, and simplify using the extra assumption that the data is standardized.  

*Answer*: $\hat{\beta}_1 = \sum_i x_i y_i$ and $\hat{\beta}_0 = 0$. 

**Part (b)**: Under the probabilistic model $y = \beta_1 x + \epsilon$, where $\epsilon \sim N(0, \sigma^2)$, show that the prediction $\hat{y} = \hat{\beta}_1 x$ is an unbiased predictor of $y$. 

*Hint:* In the standard linear regression model, $x$ is not random, only $\epsilon$ is random.

*Answer*: Using the definition of bias, $\mathbb{E}[y - \hat{y}] = \mathbb{E}[\beta_1 x + \epsilon - \hat{\beta_1} x] = \beta_1 x + 0 - x E[\hat{\beta_1}]$. Then 
\begin{align*}
\mathbb{E}[\hat{\beta}_1] &= \sum_i x_i \mathbb{E}[y_i] \\
&= \sum_i x_i (\beta_1 x_i + \mathbb{E}(\epsilon_i))
&= \sum_i \beta_1 x_i^2 \\
&= \beta_1
\end{align*}
because $\sum_i x_i^2 = 1$. Thus the bias is 0.

**Part (c)**: Why is it possible that $\hat{y}$ is unbiased is part (b), and yet linear regression models can have high bias?

*Answer*: Because the probabilistic model $y = \beta_1 x + \epsilon$ is not suitable when the actual data generating model is not linear, as is often the case in real data.

## Question 3: Model Selection

Let's return to the cars data that we looked at for HW6.

In [2]:
cars_df = pd.read_csv("https://raw.githubusercontent.com/stanford-mse-125/homework/main/data/used_cars.csv")
honda_df = cars_df[cars_df["make"] == "Honda"]

**Part (a)**: Recall from HW6 that we used polynomial regression in order to predict price from mileage for Honda cars. 

Use 5-fold cross validation to estimate the out-of-sample RMSE of the following models:

- Linear: $\widehat{\text{price}}$ = $\beta_0$ + $\beta_1$ $\cdot$ $\text{mileage}$
- Quadratic: $\widehat{\text{price}}$ = $\beta_0$ + $\beta_1$ $\cdot$ $\text{mileage}$ + $\beta_2$ $\cdot$ $\text{mileage}^2$
- Cubic: $\widehat{\text{price}}$ = $\beta_0$ + $\beta_1$ $\cdot$ $\text{mileage}$ + $\beta_2$ $\cdot$ $\text{mileage}^2$+ $\beta_3$ $\cdot$ $\text{mileage}^3$
- Quartic: $\widehat{\text{price}}$ = $\beta_0$ + $\beta_1$ $\cdot$ $\text{mileage}$ + $\beta_2$ $\cdot$ $\text{mileage}^2$ + $\beta_3$ $\cdot$ $\text{mileage}^3$ + $\beta_4$ $\cdot$ $\text{mileage}^4$

What model do you think is the best model for the data, based on the results of cross-validation?

In [36]:
# Code inspired from code in Discussion 7
# You could also do this using sklearn

# Add columns for the squared and cubed mileage
honda_df["mileage**2"] = honda_df["mileage"]**2
honda_df["mileage**3"] = honda_df["mileage"]**3
honda_df["mileage**4"] = honda_df["mileage"]**4

# Split the original data (df) into three folds after shuffling the data.
# We use three folds here since the data is small.
folds = np.array_split(honda_df, 5)

# For each held out fold, train the three linear models on the remaining
# four folds, calculate the validation RMSE on the held out fold,
# and store the results in a list.
rmse_list = []
for i in range(3):
    # get the held out fold
    val_df = folds[i]
    
    # get the other four folds
    train_df = pd.concat(folds[:i] + folds[i+1:])
    
    # train the three linear models on the training set
    linear_model = smf.ols('price ~ mileage', data=train_df).fit()
    quadratic_model = smf.ols('price ~ mileage + I(mileage**2)', data=train_df).fit()
    # cubic_model = smf.ols('price ~ mileage + I(mileage**2) + I(mileage**3)', data=train_df).fit()
    # cubic_model = sm.OLS(train_df["price"], sm.add_constant(train_df[["mileage", "mileage**2", "mileage**3"]])).fit()
    cubic_model = LinearRegression().fit(train_df[["mileage", "mileage**2", "mileage**3"]], train_df["price"])
    quartic_model = smf.ols('price ~ mileage + I(mileage**2) + I(mileage**3) + I(mileage**4)', data=train_df).fit()
    
    # calculate the validation set RMSE of the three models
    rmse_linear = np.sqrt(np.mean((val_df["price"] - linear_model.predict(val_df))**2))
    rmse_quadratic = np.sqrt(np.mean((val_df["price"] - quadratic_model.predict(val_df))**2))
    # rmse_cubic = np.sqrt(np.mean((val_df["price"] - cubic_model.predict(val_df))**2))
    rmse_cubic = np.sqrt(np.mean((val_df["price"] - cubic_model.predict(val_df[["mileage", "mileage**2", "mileage**3"]]))**2))
    # rmse_cubic = np.sqrt(np.mean((val_df["price"] - cubic_model.predict(sm.add_constant(val_df[["mileage", "mileage**2", "mileage**3"]])))**2))
    rmse_quartic = np.sqrt(np.mean((val_df["price"] - quartic_model.predict(val_df))**2))
    
    # store the results in a list
    rmse_list.append([rmse_linear, rmse_quadratic, rmse_cubic, rmse_quartic])

# Finally, average the model performance across the five folds.
# If you repeatedly run this code, you'll find that the slope+intercept model
# tends to perform best.
rmse_df = pd.DataFrame(rmse_list, columns=['linear', 'quadratic', 'cubic', 'quartic'])
rmse_df.mean()

/var/folders/ty/g5x2zyjs7jn1pbfnvwkvwh7w0000gn/T/ipykernel_58922/1310635668.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  honda_df["mileage**2"] = honda_df["mileage"]**2
/var/folders/ty/g5x2zyjs7jn1pbfnvwkvwh7w0000gn/T/ipykernel_58922/1310635668.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  honda_df["mileage**3"] = honda_df["mileage"]**3
/var/folders/ty/g5x2zyjs7jn1pbfnvwkvwh7w0000gn/T/ipykernel_58922/1310635668.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

linear       2328.250723
quadratic    2256.255468
cubic        2259.343067
quartic      5929.687497
dtype: float64

In [27]:
LinearRegression().fit(train_df[["mileage"]], train_df["price"]).coef_

array([-0.07854082])

In [41]:
# model = smf.ols('price ~ mileage + I(mileage**2) + I(mileage**3)', data=honda_df).fit()
model = sm.OLS(honda_df["price"], sm.add_constant(honda_df[["mileage", "mileage**2", "mileage**3"]])).fit()
print(mean_squared_error(honda_df["price"], model.predict(sm.add_constant(honda_df[["mileage", "mileage**2", "mileage**3"]])), squared=False))
model.summary()

5653.292985850861


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                      -0.617
Model:                            OLS   Adj. R-squared:                 -0.631
Method:                 Least Squares   F-statistic:                    -44.05
Date:                Thu, 01 Jun 2023   Prob (F-statistic):               1.00
Time:                        22:32:39   Log-Likelihood:                -2353.8
No. Observations:                 234   AIC:                             4714.
Df Residuals:                     231   BIC:                             4724.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.259e-05   7.73e-07     29.212      0.000    2.11e-05    2.41e-05
mileage        0.6289      0.022     29.212      0.000       0.587       0.671
mileage**2 -6.589e-06    3.1e-07    -21.227      0.000    -7.2e-06   -5.98e-06
mileage**3   1.69e-11   1.03e-12     16.449      0.000    1.49e-11    1.89e-11
==============================================================================
Omnibus:                       15.349   Durbin-Watson:                   1.781
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               18.182
Skew:                           0.521   Prob(JB):                     0.000113
Kurtosis:                       3.882   Cond. No.                     8.31e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.31e+15. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
honda_df[["mileage", "mileage**2", "mileage**3"]]

In [31]:
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(honda_df[["mileage", "mileage**2", "mileage**3"]], honda_df["price"])
print(model.coef_)
mean_squared_error(honda_df["price"], model.predict(honda_df[["mileage", "mileage**2", "mileage**3"]]), squared=False)

[-8.17880660e-02 -2.04675498e-07  1.18177229e-12]


2205.499190151544

In [13]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from patsy import dmatrix
from sklearn.pipeline import make_pipeline

X = honda_df[["mileage"]]
# X_cubic = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X)

pipeline = make_pipeline(
    PolynomialFeatures(degree=3, include_bias=False),
    LinearRegression(), 
)

np.mean(-1 * cross_val_score(pipeline, X, honda_df["price"], cv=5, scoring="neg_root_mean_squared_error"))

2316.3546127958566

The quadratic model looks like the best model. The cubic and quartic models seem to severely overfit the training data, while the quadratic model narrowly beats the linear model.

**Part (b)**: Repeat part (a), but for each linear model, also add in terms for $\text{year}$ and $\text{model}$ of the Honda. Do the results change? If so, how?

In [12]:
# Code inspired from code in Discussion 7
# You could also do this using sklearn

# Split the original data (df) into three folds after shuffling the data.
# We use three folds here since the data is small.
folds = np.array_split(honda_df, 5)

# For each held out fold, train the three linear models on the remaining
# four folds, calculate the validation RMSE on the held out fold,
# and store the results in a list.
rmse_list = []
for i in range(5):
    # get the held out fold
    val_df = folds[i]
    
    # get the other four folds
    train_df = pd.concat(folds[:i] + folds[i+1:])
    
    # train the three linear models on the training set
    linear_model = smf.ols('price ~ mileage + year + model', data=train_df).fit()
    quadratic_model = smf.ols('price ~ mileage + year + model + I(mileage**2)', data=train_df).fit()
    cubic_model = smf.ols('price ~ mileage + year + model + I(mileage**2) + I(mileage**3)', data=train_df).fit()
    quartic_model = smf.ols('price ~ mileage + year + model + I(mileage**2) + I(mileage**3) + I(mileage**4)', data=train_df).fit()
    
    # calculate the validation set RMSE of the three models
    rmse_linear = np.sqrt(np.mean((val_df["price"] - linear_model.predict(val_df))**2))
    rmse_quadratic = np.sqrt(np.mean((val_df["price"] - quadratic_model.predict(val_df))**2))
    rmse_cubic = np.sqrt(np.mean((val_df["price"] - cubic_model.predict(val_df))**2))
    rmse_quartic = np.sqrt(np.mean((val_df["price"] - quartic_model.predict(val_df))**2))
    
    # store the results in a list
    rmse_list.append([rmse_linear, rmse_quadratic, rmse_cubic, rmse_quartic])

# Finally, average the model performance across the five folds.
# If you repeatedly run this code, you'll find that the slope+intercept model
# tends to perform best.
rmse_df = pd.DataFrame(rmse_list, columns=['linear', 'quadratic', 'cubic', 'quartic'])
rmse_df.mean()

linear       1748.790013
quadratic    1703.385754
cubic        2251.383454
quartic      5891.859023
dtype: float64

The quadratic model is still the best model. Interestingly, the cubic model is now way better than the quartic model (which was not the case in part (a)), but still worse than the linear and quadratic models. 